In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Luke\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# Reading the News API key enviroment variable

api_key = '755bee183ad0447587d1d607d884ca99'

In [5]:
# Create a newsapi client

newsapi = NewsApiClient(api_key=api_key)

In [23]:
# Fetch all the news about Desktop Metals
dm = newsapi.get_everything(
    q="Apple",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles Desktop Metals: {dm['totalResults']}")

# Show sample article
dm["articles"][0]

Total articles Desktop Metals: 34530


{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'Nathan Ingraham',
 'title': 'T-Mobile is giving customers a free year of Apple TV+',
 'description': "For a few years now, wireless carriers in the US have offered their customers all manner of video and music freebies. Verizon (Engadget's parent company) has offered free subscriptions to Disney+, Apple Music and AMC+ recently, while T-Mobile has long offered…",
 'url': 'https://www.engadget.com/t-mobile-apple-tv-plus-free-year-subscription-150820950.html',
 'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2021-08/7de29310-0423-11ec-9fd7-da06f23899fc',
 'publishedAt': '2021-08-23T15:08:20Z',
 'content': "For a few years now, wireless carriers in the US have offered their customers all manner of video and music freebies. Verizon (Engadget's parent company) has offered free subscriptions to Disney+, Ap… [+1342 chars]"}

In [24]:
# Create the Desktop Metals sentiment scores DataFrame
dm_sentiments = []

for article in dm["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        dm_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
dm_df = pd.DataFrame(dm_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
dm_df = dm_df[cols]

dm_df.head()

,date,text,compound,positive,negative,neutral
0,2021-08-23,"For a few years now, wireless carriers in the ...",0.7269,0.164,0.000,0.836
1,2021-08-12,It looks like HomePod\r\n and HomePod mini\r\n...,0.6369,0.136,0.000,0.864
2,2021-08-13,Apple is ready to acknowledge the controversy ...,0.3453,0.201,0.097,0.702
3,2021-07-30,"For at least a decade, privacy advocates dream...",0.3400,0.093,0.000,0.907
4,2021-08-15,Apple has said it plans to fight the decision\...,-0.4588,0.000,0.105,0.895


In [25]:
# Get descriptive stats from the DataFrame
dm_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.109805,0.070470,0.045560,0.883950
std,0.467882,0.075147,0.070196,0.095215
min,-0.893400,0.000000,0.000000,0.636000
25%,-0.102700,0.000000,0.000000,0.826750
50%,0.077200,0.060500,0.000000,0.897000
75%,0.462300,0.108500,0.073750,0.959000
max,0.888500,0.333000,0.312000,1.000000
